In [95]:
import pandas as pd
import numpy as np
import os
import re
import pickle
from bs4 import BeautifulSoup as soup
import requests as rq
import time

# EDA

## Pulling in Data

The data collected is a mix of HDF5, csv and web scraped data about the Bay area.  The CSVs are for particle concentraions, HDF5s for CO2 ppm, and the web scraped data is for empty lot/apartments for sale in the San Fransisco.

In [73]:
#starting off by reading in the csv files
aqcsvs = []
aqcsvs += ['airqualitycsvs/'+each for each in os.listdir('airqualitycsvs/') if each.endswith('.csv')]


#getting all H5 files for the project
results = []
results += ['../../../../Downloads/'+each for each in os.listdir('../../../../Downloads/') if each.endswith('.h5')]

In [74]:
aqcsvs

['airqualitycsvs/annual_conc_by_monitor_2018.csv',
 'airqualitycsvs/annual_conc_by_monitor_2017.csv',
 'airqualitycsvs/annual_conc_by_monitor_2016.csv',
 'airqualitycsvs/annual_conc_by_monitor_2014.csv',
 'airqualitycsvs/annual_conc_by_monitor_2015.csv']

In [80]:
aq= pd.read_csv(aqcsvs[0])
for i in range(1,len(aqcsvs)):
    aq.append(pd.read_csv(aqcsvs[i]))

In [82]:
#Filtering out unneed data
aq = aq[aq['State Name']=='California']
aq = aq[aq['CBSA Name']=='San Francisco-Oakland-Hayward, CA']
aq.reset_index(drop=True,inplace=True)
aq.drop(['State Code', 'County Code','State Name','City Name','County Name', 'Datum'], axis=1)

In [89]:
aq.head()

State Code  County Code  Site Num  Parameter Code  POC   Latitude  \
0           6            1         7           42601    1  37.687526   
1           6            1         7           42602    1  37.687526   
2           6            1         7           42602    1  37.687526   
3           6            1         7           42603    1  37.687526   
4           6            1         7           43207    3  37.687526   

    Longitude  Datum            Parameter Name Sample Duration  \
0 -121.784217  WGS84         Nitric oxide (NO)          1 HOUR   
1 -121.784217  WGS84    Nitrogen dioxide (NO2)          1 HOUR   
2 -121.784217  WGS84    Nitrogen dioxide (NO2)          1 HOUR   
3 -121.784217  WGS84  Oxides of nitrogen (NOx)          1 HOUR   
4 -121.784217  WGS84                 Freon 113         24 HOUR   

          ...          75th Percentile 50th Percentile 10th Percentile  \
0         ...                    7.900           1.100          -0.100   
1         ...                   28.200          24.500          14.600   
2         ...                   18.700          10.200           2.800   
3         ...                   28.700          11.600           3.000   
4         ...                    0.148           0.144           0.128   

   Local Site Name          Address  State Name  County Name  City Name  \
0        Livermore  793 Rincon Ave.  California      Alameda  Livermore   
1        Livermore  793 Rincon Ave.  California      Alameda  Livermore   
2        Livermore  793 Rincon Ave.  California      Alameda  Livermore   
3        Livermore  793 Rincon Ave.  California      Alameda  Livermore   
4        Livermore  793 Rincon Ave.  California      Alameda  Livermore   

                           CBSA Name  Date of Last Change  
0  San Francisco-Oakland-Hayward, CA           2018-04-04  
1  San Francisco-Oakland-Hayward, CA           2018-04-04  
2  San Francisco-Oakland-Hayward, CA           2018-04-04  
3  San Francisco-Oakland-Hayward, CA           2018-04-04  
4  San Francisco-Oakland-Hayward, CA           2018-05-10  

[5 rows x 55 columns]

In [102]:
def h5ToDF(file):
    f = h5py.File(file, 'r')
    date = [pd.to_datetime(str(x).replace('b\'','').replace('\'','')) for x in list(f['RetrievalHeader']['retrieval_time_string'])]
    latlong = [(x,y) for x,y in zip(f['RetrievalGeometry']['retrieval_latitude_geoid'],f['RetrievalGeometry']['retrieval_longitude_geoid'])]
    co2ppm = list(f['RetrievalResults']['xco2'])
    xco2_uncert = list(f['RetrievalResults']['xco2_uncert'])
    df = pd.DataFrame({'date':date,
        'latlong': latlong,
        'co2ppm': co2ppm,
        'xco2_uncert': xco2_uncert
    })
    return df
                                                                                    
                                                                                    
                                                                                    

In [ ]:
df = f = h5py.File(results[0], 'r')
date = [pd.to_datetime(str(x).replace('b\'','').replace('\'','')) for x in list(f['RetrievalHeader']['retrieval_time_string'])]
latlong = [(x,y) for x,y in zip(f['RetrievalGeometry']['retrieval_latitude_geoid'],f['RetrievalGeometry']['retrieval_longitude_geoid'])]
co2ppm = list(f['RetrievalResults']['xco2'])
xco2_uncert = list(f['RetrievalResults']['xco2_uncert'])
df = pd.DataFrame({'date':date,
                'latlong': latlong,
                'co2ppm': co2ppm,
                'xco2_uncert': xco2_uncert})
for i in range(1,len(results)):
    df.append(h5ToDF(results[i]))

In [ ]:
df.head()

In [ ]:
#Writing a pickle file to save data
import pickle
with open('my_data.pkl', 'wb') as picklefile:
    pickle.dump(df, picklefile)

In [ ]:
#list(f['RetrievalGeometry']['retrieval_latitude_geoid'])

## Web Scraping

Pulling data from loopnet.com for areas where we can greenify citys (make them more environmentally friendly). 

In [97]:
#initializing sites that I will pull from
flexspace = 'http://www.loopnet.com/california/san-francisco_flex-space-for-sale/'
land = 'http://www.loopnet.com/california/san-francisco_land-for-sale/'
apts = 'http://www.loopnet.com/california/san-francisco_apartment-buildings-for-sale/'

In [98]:
response = rq.get(land)
page = response.text
reviewp = soup(page,"lxml")


In [99]:
reviewp.find_all(class_='listingTitle')

[<span class="listingTitle">0.05 Acre Site for Proposed Apartments</span>,
 <span class="listingTitle">1228 Folsom Street</span>,
 <span class="listingTitle">1876 Oakdale Ave</span>,
 <span class="listingTitle">1791 Mission Street</span>,
 <span class="listingTitle">240 7th St</span>,
 <span class="listingTitle">1924 Mission St</span>,
 <span class="listingTitle">2435-2445 16th St</span>,
 <span class="listingTitle">Topaz Way</span>,
 <span class="listingTitle">Portfolio of 4 Properties</span>,
 <span class="listingTitle">1234 Shoreline Hwy</span>,
 <span class="listingTitle">81 Ervine St</span>,
 <span class="listingTitle">OCTAVIA M+N</span>,
 <span class="listingTitle">250 Church St</span>,
 <span class="listingTitle">Mission and 22nd</span>]